In [1]:
#%pip uninstall stable-baselines3
#%pip install stable-baselines3==1.7.0 --no-deps

In [2]:
import os
from stable_baselines3 import PPO
from stable_baselines3 import DQN
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecTransposeImage
from matplotlib import pyplot as plt
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT,COMPLEX_MOVEMENT,RIGHT_ONLY
import torch

In [3]:
def createEnvironments_DQN(env_name):
    env = gym_super_mario_bros.make(env_name)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    env = GrayScaleObservation(env, keep_dim=True)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env,4,channels_order='last')
    env = VecTransposeImage(env)
    return env

In [4]:
def createEnvironment_PPO(env_name):
        env = gym_super_mario_bros.make(env_name)
        env = JoypadSpace(env, SIMPLE_MOVEMENT)
        env = GrayScaleObservation(env, keep_dim=True)
        return env
    
def allWorldStages():
    worlds = [1, 2, 3, 4, 5, 6, 7, 8]
    #Training on Stages 1-3 for each world, testing on final stage of each world
    stages = [4]

    for world in worlds:
        for stage in stages:
            environment_name = f"SuperMarioBros-{world}-{stage}-v0"
            yield environment_name

In [5]:
env_names = [name for name in allWorldStages()]
env_names = [1,2,3,4,5,6,7,8,9,10]
#print(env_names)

env = DummyVecEnv([lambda env_name=name: createEnvironment_PPO("SuperMarioBros-8-4-v0") for name in env_names])
env = VecTransposeImage(env)

observation_space = env.observation_space
action_space = env.action_space
model = PPO.load("./train/FinalModels/PPO_10",env,custom_objects={'observation_space': observation_space, 'action_space': action_space})

c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym\envs\registration.py:593: UserWarning: WARN: The environment SuperMarioBros-8-4-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
state = env.reset()
episode_reward = 0
episode_score = 0
episode_global_dist = 0
cum_reward = 0
count =0
while count <= 100:
    episode_global_dist = 0
    action, _state, = model.predict(state)
    state, reward, done, info = env.step(action)
    episode_reward =+ reward
    game_info = info[0]
    episode_score =+ game_info['score']
    episode_dist =+ game_info['x_pos']
    if episode_dist is not None:       
        episode_furthest_distance = max(episode_global_dist, episode_dist)
        episode_global_dist = max(episode_global_dist, episode_dist)
    cum_reward += reward
    if any(done):
        count = count + 1
        print("Episode" , count)
        print("Cumulative Reward" , cum_reward)
        print(f"Reward {episode_reward}. Score {episode_score}. Distance {episode_global_dist}")
        #print(f"Episode furthest distance:{episode_furthest_distance}. Episode global distance {episode_global_dist}")
    env.render()
env.close()